In [0]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import psycopg2
from psycopg2 import sql
from datetime import datetime

In [0]:
def scrape_coinmarketcap():
    # Empty lists to store the data
    names_list = []
    prices_list = []
    percent_1h_list = []
    percent_24h_list = []
    percent_7d_list = []
    market_caps_list = []
    volumes_24h_list = []

In [0]:
names_list = []
prices_list = []
percent_1h_list = []
percent_24h_list = []
percent_7d_list = []
market_caps_list = []
volumes_24h_list = []

# Send request and get the response
url = 'https://coinmarketcap.com/'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'}
response = requests.get(url, headers=headers)
response.raise_for_status()  # Raise an error for bad responses

# Parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table that contains the data
table = soup.find('table', attrs={'class': 'cmc-table'})

# Find all rows in the table
rows = table.find_all('tr')

# Skip the first row (header) and loop through the remaining rows
for row in rows[1:11]:  # Limiting to the first 10 rows
    # Extracting data from each cell in the row
    try:
        name_elem = row.find('p', attrs={'class': 'sc-65e7f566-0 iPbTJf coin-item-name'})
        price_elem = row.find('div', attrs={'class': 'sc-b3fc6b7-0 dzgUIj'})
        percent_1h_elem = row.find_all('span', attrs={'class': 'sc-a59753b0-0 cmnujh'})  # Update based on actual class names
        percent_24h_elem = row.find_all('span', attrs={'class': 'sc-a59753b0-0 cmnujh'})  # Update based on actual class names
        percent_7d_elem = row.find_all('span', attrs={'class': 'sc-a59753b0-0 cmnujh'})  # Update based on actual class names
        market_cap_elem = row.find('span', attrs={'class': 'sc-11478e5d-0 chpohi'})  # Update based on actual class names
        volume_24h_elem = row.find('p', attrs={'class': 'sc-71024e3e-0 bbHOdE font_weight_500'})  # Update based on actual class names

        # Check if elements are found before accessing .text
        name = name_elem.text.strip() if name_elem else "N/A"
        price = price_elem.text.strip() if price_elem else "N/A"
        percent_1h = percent_1h_elem[0].text.strip() if len(percent_1h_elem) > 0 else "N/A"
        percent_24h = percent_24h_elem[1].text.strip() if len(percent_24h_elem) > 1 else "N/A"
        percent_7d = percent_7d_elem[2].text.strip() if len(percent_7d_elem) > 2 else "N/A"
        market_cap = market_cap_elem.text.strip() if market_cap_elem else "N/A"
        volume_24h = volume_24h_elem.text.strip() if volume_24h_elem else "N/A"

        # Append the data into lists
        names_list.append(name)
        prices_list.append(price)
        percent_1h_list.append(percent_1h)
        percent_24h_list.append(percent_24h)
        percent_7d_list.append(percent_7d)
        market_caps_list.append(market_cap)
        volumes_24h_list.append(volume_24h)

    except Exception as e:  # General exception handling for unexpected errors
        print(f"Error extracting data from row: {e}")
        continue

# Create DataFrame
df = pd.DataFrame({
    'Name': names_list,
    'Price': prices_list,
    'Percent Change 1H': percent_1h_list,
    'Percent Change 24H': percent_24h_list,
    'Percent Change 7D': percent_7d_list,
    'Market Cap': market_caps_list,
    'Volume 24H': volumes_24h_list
})

# Replace "N/A" with np.nan, then fill NaN values with 0
df.replace("N/A", np.nan, inplace=True)
df.fillna(0, inplace=True)

In [0]:
df.head(10)

,Name,Price,Percent Change 1H,Percent Change 24H,Percent Change 7D,Market Cap,Volume 24H
0,Bitcoin,"$64,095.59",0.63%,0.71%,2.65%,$1.27T,"$24,691,829,672"
1,Ethereum,"$2,629.80",0.49%,0.11%,8.18%,$316.49B,"$14,512,490,981"
2,Tether,$0.9999,0.01%,0,0,$119.27B,"$50,954,924,510"
3,BNB,$595.49,0.26%,7.06%,0,$86.8B,"$1,799,597,108"
4,Solana,$151.32,0.49%,0.69%,8.93%,$70.94B,"$1,933,071,312"
5,USDC,$0.9999,0.00%,0.01%,0,$36.08B,"$5,126,861,211"
6,XRP,$0.589,0.35%,0.13%,0.92%,$33.28B,"$1,037,832,420"
7,Dogecoin,$0.1143,0.84%,4.92%,9.33%,$16.7B,"$946,018,603"
8,Toncoin,$5.81,0.23%,2.12%,1.09%,$14.73B,"$265,943,439"
9,Cardano,$0.3942,0.24%,2.90%,13.21%,$13.77B,"$301,992,406"


In [0]:
df['Price'] = df['Price'].replace({'\$': '', ',': ''}, regex=True).astype(float)
df['Percent Change 1H'] = df['Percent Change 1H'].replace({'%': ''}, regex=True).astype(float)
df['Percent Change 24H'] = df['Percent Change 24H'].replace({'%': ''}, regex=True).astype(float)
df['Percent Change 7D'] = df['Percent Change 7D'].replace({'%': ''}, regex=True).astype(float)
df['Market Cap'] = df['Market Cap'].replace({'\$': '', ',': '', 'T': 'e12', 'B': 'e9'}, regex=True).astype(float)
df['Volume 24H'] = df['Volume 24H'].replace({'\$': '', ',': ''}, regex=True).astype(float)



In [0]:
# Display the cleaned DataFrame
df.head()

,Name,Price,Percent Change 1H,Percent Change 24H,Percent Change 7D,Market Cap,Volume 24H
0,Bitcoin,64095.5900,0.63,0.71,2.65,1.270000e+12,2.469183e+10
1,Ethereum,2629.8000,0.49,0.11,8.18,3.164900e+11,1.451249e+10
2,Tether,0.9999,0.01,0.00,0.00,1.192700e+11,5.095492e+10
3,BNB,595.4900,0.26,7.06,0.00,8.680000e+10,1.799597e+09
4,Solana,151.3200,0.49,0.69,8.93,7.094000e+10,1.933071e+09


In [0]:
df.rename(columns={"Price": "Price($)"}, inplace=True)
df.rename(columns={"Market Cap": "Market Cap($)"}, inplace=True)
df.rename(columns={"Volume 24H": "Volume 24H($)"}, inplace=True)

In [0]:
df.head()

,Name,Price($),Percent Change 1H,Percent Change 24H,Percent Change 7D,Market Cap($),Volume 24H($)
0,Bitcoin,64095.5900,0.63,0.71,2.65,1.270000e+12,2.469183e+10
1,Ethereum,2629.8000,0.49,0.11,8.18,3.164900e+11,1.451249e+10
2,Tether,0.9999,0.01,0.00,0.00,1.192700e+11,5.095492e+10
3,BNB,595.4900,0.26,7.06,0.00,8.680000e+10,1.799597e+09
4,Solana,151.3200,0.49,0.69,8.93,7.094000e+10,1.933071e+09


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Name                10 non-null     object 
 1   Price($)            10 non-null     float64
 2   Percent Change 1H   10 non-null     float64
 3   Percent Change 24H  10 non-null     float64
 4   Percent Change 7D   10 non-null     float64
 5   Market Cap($)       10 non-null     float64
 6   Volume 24H($)       10 non-null     float64
dtypes: float64(6), object(1)
memory usage: 688.0+ bytes


In [0]:
df.head()

,Name,Price($),Percent Change 1H,Percent Change 24H,Percent Change 7D,Market Cap($),Volume 24H($)
0,Bitcoin,64095.5900,0.63,0.71,2.65,1.270000e+12,2.469183e+10
1,Ethereum,2629.8000,0.49,0.11,8.18,3.164900e+11,1.451249e+10
2,Tether,0.9999,0.01,0.00,0.00,1.192700e+11,5.095492e+10
3,BNB,595.4900,0.26,7.06,0.00,8.680000e+10,1.799597e+09
4,Solana,151.3200,0.49,0.69,8.93,7.094000e+10,1.933071e+09


In [0]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType
from pyspark.sql.functions import col, regexp_replace, sum, avg
from pyspark.sql.functions import *


In [0]:
spark = SparkSession.builder.appName("coinmarketcap").getOrCreate()

In [0]:
data = pd.DataFrame(df)
spark_df = spark.createDataFrame(data)

In [0]:
# Register as a SQL temporary view
spark_df.createOrReplaceTempView("coinmarketcap")

In [0]:
result = spark.sql("SELECT * FROM coinmarketcap LIMIT 10")
result.show()

+--------+--------+-----------------+------------------+-----------------+-------------+---------------+
|    Name|Price($)|Percent Change 1H|Percent Change 24H|Percent Change 7D|Market Cap($)|  Volume 24H($)|
+--------+--------+-----------------+------------------+-----------------+-------------+---------------+
| Bitcoin|64095.59|             0.63|              0.71|             2.65|      1.27E12|2.4691829672E10|
|Ethereum|  2629.8|             0.49|              0.11|             8.18|    3.1649E11|1.4512490981E10|
|  Tether|  0.9999|             0.01|               0.0|              0.0|    1.1927E11| 5.095492451E10|
|     BNB|  595.49|             0.26|              7.06|              0.0|      8.68E10|  1.799597108E9|
|  Solana|  151.32|             0.49|              0.69|             8.93|     7.094E10|  1.933071312E9|
|    USDC|  0.9999|              0.0|              0.01|              0.0|     3.608E10|  5.126861211E9|
|     XRP|   0.589|             0.35|              0.13

In [0]:
spark.sql("DESCRIBE coinmarketcap").show()


+------------------+---------+-------+
|          col_name|data_type|comment|
+------------------+---------+-------+
|              Name|   string|   null|
|          Price($)|   double|   null|
| Percent Change 1H|   double|   null|
|Percent Change 24H|   double|   null|
| Percent Change 7D|   double|   null|
|     Market Cap($)|   double|   null|
|     Volume 24H($)|   double|   null|
+------------------+---------+-------+



In [0]:
%sql
SELECT Name, "Market Cap($)"
FROM coinmarketcap
ORDER BY "Market Cap($)" DESC
LIMIT 3;


Name,Market Cap($)
Bitcoin,Market Cap($)
Ethereum,Market Cap($)
Tether,Market Cap($)


In [0]:
%sql
SELECT AVG(`Percent Change 24H`) AS `Average Percent Change 24H`
FROM coinmarketcap;

Average Percent Change 24H
1.8649999999999998


In [0]:
%sql
SELECT SUM(`Market Cap($)`) AS `Total Market Cap`
FROM coinmarketcap
WHERE `Volume 24H($)` > 1e9

Total Market Cap
1.93286E12


In [0]:
%sql
SELECT Name, `Price($)`
FROM coinmarketcap
ORDER BY `Price($)` DESC
LIMIT 1;

Name,Price($)
Bitcoin,64095.59


In [0]:
%sql
SELECT Name, "Percent Change 1H", "Percent Change 24H"
FROM coinmarketcap
WHERE "Percent Change 1H" > 0 AND "Percent Change 24H" > 0;


Name,Percent Change 1H,Percent Change 24H


In [0]:
%sql
SELECT SUM(`Volume 24H($)`) AS `Total Volume`
FROM coinmarketcap
WHERE `Market Cap($)` > 1e11;

Total Volume
9.0159245163E10


In [0]:
%pip install sqlalchemy

Python interpreter will be restarted.
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-9ad89c65-aa1c-40c0-9d0b-67668e60399a
    Can't uninstall 'typing-extensions'. No files were found to uninstall.
Python interpreter will be restarted.


In [0]:
from sqlalchemy import create_engine

In [0]:
%sql
create database if not exists project

In [0]:
database_name = 'project'

In [0]:
# Example of loading data into a DataFrame
spark_df = spark.read.table("coinmarketcap")


In [0]:
# Example: Replacing invalid characters
clean_columns = [col.replace(" ", "_").replace(",", "").replace(";", "").replace("(", "").replace(")", "") for col in spark_df.columns]
spark_df = spark_df.toDF(*clean_columns)


In [0]:
# Replace 'database_name' with your actual database name
spark_df.write.mode("overwrite").saveAsTable("project.coinmarketcap")


In [0]:
%sql
select * from coinmarketcap;

Name,Price($),Percent Change 1H,Percent Change 24H,Percent Change 7D,Market Cap($),Volume 24H($)
Bitcoin,64095.59,0.63,0.71,2.65,1.27E12,2.4691829672E10
Ethereum,2629.8,0.49,0.11,8.18,3.1649E11,1.4512490981E10
Tether,0.9999,0.01,0.0,0.0,1.1927E11,5.095492451E10
BNB,595.49,0.26,7.06,0.0,8.68E10,1.799597108E9
Solana,151.32,0.49,0.69,8.93,7.094E10,1.933071312E9
USDC,0.9999,0.0,0.01,0.0,3.608E10,5.126861211E9
XRP,0.589,0.35,0.13,0.92,3.328E10,1.03783242E9
Dogecoin,0.1143,0.84,4.92,9.33,1.67E10,9.46018603E8
Toncoin,5.81,0.23,2.12,1.09,1.473E10,2.65943439E8
Cardano,0.3942,0.24,2.9,13.21,1.377E10,3.01992406E8
